In [1]:
import pandas as pd
import glob
import re
import os

In [2]:
def read_and_parse(file_path):
    """
    Lê um CSV no formato:
      - 1ª linha de cabeçalho (pulada)
      - Linhas: <State0 ... StateN>  Time  Trial
      - Possíveis linhas em branco
    Retorna DataFrame com colunas State0..StateN, Time e Trial.
    """
    df = pd.read_csv(
        file_path,
        sep=r"\s+|\t+",
        engine="python",
        header=None,
        skiprows=1,
        skip_blank_lines=True
    ).dropna(how='all')
    
    total_cols = df.shape[1]
    n_states = total_cols - 2
    state_cols = [f"State{i}" for i in range(n_states)]
    df.columns = state_cols + ["Time", "Trial"]
    return df


In [3]:
def average_trials_all():
    """
    Para cada prefixo <namebase> (ex: 'Beta1') encontrado em arquivos
    Beta1Data0.csv, Beta1Data1.csv, …,
    faz a média das trials por Time e salva Beta1Data_avg.csv.
    """
    pattern = re.compile(r"(.+?)Data\d+\.csv$")
    groups = {}
    for path in glob.glob("*.csv"):
        m = pattern.match(path)
        if m:
            prefix = m.group(1)
            groups.setdefault(prefix, []).append(path)
    
    if not groups:
        print("⚠️ Nenhum arquivo '*DataN.csv' encontrado.")
        return
    
    for prefix, files in groups.items():
        print(f"Processando '{prefix}': {len(files)} arquivos")
        dfs = [read_and_parse(f) for f in sorted(files)]
        all_df = pd.concat(dfs, ignore_index=True)
        avg_df = all_df.groupby("Time", as_index=False).mean()
        out_name = f"{prefix}Data_avg.csv"
        avg_df.to_csv(out_name, index=False)
        print(f"→ Salvo: {out_name}")


In [4]:
if __name__ == "__main__":
    average_trials_all()


Processando 'Delta10': 7 arquivos
→ Salvo: Delta10Data_avg.csv
Processando 'Delta11': 7 arquivos
→ Salvo: Delta11Data_avg.csv
Processando 'Delta1': 7 arquivos
→ Salvo: Delta1Data_avg.csv
Processando 'Delta2': 7 arquivos
→ Salvo: Delta2Data_avg.csv
Processando 'Delta3': 7 arquivos
→ Salvo: Delta3Data_avg.csv
Processando 'Delta4': 7 arquivos
→ Salvo: Delta4Data_avg.csv
Processando 'Delta5': 7 arquivos
→ Salvo: Delta5Data_avg.csv
Processando 'Delta6': 7 arquivos
→ Salvo: Delta6Data_avg.csv
Processando 'Delta7': 7 arquivos
→ Salvo: Delta7Data_avg.csv
Processando 'Delta8': 7 arquivos
→ Salvo: Delta8Data_avg.csv
Processando 'Delta9': 7 arquivos
→ Salvo: Delta9Data_avg.csv
